In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd drive/MyDrive/projects/OCOR

/content/drive/MyDrive/projects/OCOR


验证当前环境：

In [3]:
!pwd
# !pip list

import torch, mmcv
print(torch.__version__)         # eg. 2.0.1
print(torch.version.cuda)        # eg. 11.8
print(mmcv.__version__)

/content


ModuleNotFoundError: No module named 'mmcv'

In [6]:
# ========================== 1. 使用 GPU ==========================
# 如果你不是 GPU 环境，点击上方菜单：修改 > 笔记本设置 > 硬件加速器 选 GPU
!nvidia-smi


/bin/bash: line 1: nvidia-smi: command not found


In [ ]:

# ========================== 2. 克隆 OCOR 项目 ==========================
# !git clone https://github.com/AliasJeff/OCOR.git
# %cd OCOR
# %ls


Cloning into 'OCOR'...
remote: Enumerating objects: 1093, done.
remote: Counting objects: 100% (972/972), done.
remote: Compressing objects: 100% (603/603), done.
remote: Total 1093 (delta 382), reused 941 (delta 369), pack-reused 121 (from 1)
Receiving objects: 100% (1093/1093), 7.40 MiB | 13.53 MiB/s, done.
Resolving deltas: 100% (389/389), done.
/content/OCOR
configs/        docs/            mmcv_custom/   requirements.txt  tests/
DatasetTest.py  evaluate_SOR.py  mmdet/         resources/        tools/
demo/           inference.py     README.md      setup.cfg         utils.py
docker/         LICENSE          requirements/  setup.py


In [ ]:
!pip uninstall -y numpy

# 安装 NumPy 1.x（避免 ABI 问题）
!pip install numpy==1.24.3


Found existing installation: numpy 1.24.3
Uninstalling numpy-1.24.3:
  Successfully uninstalled numpy-1.24.3
  Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
xarray-einstats 0.9.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
blosc2 3.5.1 requires numpy>=1.26, but you have numpy 1.24.3 which is incompatible.
yfinance 0.2.65 requires requests>=2.31, but you have requests 2.28.2 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
pymc 5.23.0 requires numpy>=1.25.0, but you have numpy 1.24.3 which is incompatible.
pymc 5.23.0 requires rich>=13.7.1, but you have rich 13.4.2 which is incompat

In [ ]:

# ========================== 3. 安装依赖 ==========================

# Step 1: 清理旧依赖
!pip uninstall -y mmcv mmcv-full mmdet torch torchvision

# Step 2: 安装 PyTorch 2.0.1 + CUDA 11.8（与 mmcv-full 兼容）
!pip install torch==2.0.1 torchvision==0.15.2 --index-url https://download.pytorch.org/whl/cu118

# Step 3: 安装 mmcv-full 1.7.1（适配 mmdet 2.12.0 和 torch 2.0）
# !pip install mmcv==1.3.9 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.0/index.html
!pip install mmcv-full==1.7.2 -f https://download.openmmlab.com/mmcv/dist/cu118/torch2.0/index.html

# Step 4: 安装 MMDetection 2.12.0
!pip install mmdet==2.12.0

# 安装 ninja
!pip install ninja

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 848.7/8

In [ ]:
# 安装 mmdet + 本地项目（OCOR）
!pip install -e .

Obtaining file:///content/drive/MyDrive/projects/OCOR
  Preparing metadata (setup.py) ... done
  Attempting uninstall: mmdet
    Found existing installation: mmdet 2.12.0
    Uninstalling mmdet-2.12.0:
      Successfully uninstalled mmdet-2.12.0
  Running setup.py develop for mmdet


In [ ]:


# ========================== 4. 下载预训练 Swin 模型 ==========================
!mkdir -p checkpoints
!wget https://download.openmmlab.com/mmclassification/v0/swin-transformer/swin-large_3rdparty_in21k_20220301-040dd71c.pth -O checkpoints/swin-large.pth


--2025-07-15 15:47:25--  https://download.openmmlab.com/mmclassification/v0/swin-transformer/swin-large_3rdparty_in21k_20220301-040dd71c.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.246.23.180, 47.246.23.183, 47.246.23.177, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|47.246.23.180|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-07-15 15:47:26 ERROR 404: Not Found.



In [ ]:
!python setup.py develop

running develop
/usr/local/lib/python3.11/dist-packages/setuptools/command/easy_install.py:156: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running egg_info
writing manifest file 'mmdet.egg-info/SOURCES.txt'
running build_ext


In [ ]:

# ========================== 5. 启动训练 ==========================
# 如果你有自定义数据目录，记得修改 config 文件中的数据路径
!python tools/train.py configs/ocor/ocor_swin_large_patch4_window7_fpn_300_proposals.py


/usr/local/lib/python3.11/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
apex is not installed
/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
apex is not installed
apex is not installed
apex is not installed
fatal: not a git repository (or any parent up to mount point /content)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
2025-07-16 01:00:03,183 - mmdet - INFO - Environment info:
---------

In [ ]:

# ========================== 6. 将训练结果保存到 Google Drive（可选） ==========================
from google.colab import drive
drive.mount('/content/drive')

# 将训练权重保存到 Google Drive
!cp work_dirs/ocor_swin_large_patch4_window7_fpn_300_proposals/latest.pth /content/drive/MyDrive/